In [ ]:
# -*- coding: utf-8 -*-

# This code is part of Qiskit.
#
# (C) Copyright IBM 2017, 2021.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

In [ ]:
import qiskit_metal as metal
import numpy as np
from math import *
from qiskit_metal import designs, draw, Dict
from qiskit_metal.qlibrary.core import BaseQubit, QComponent
from qiskit_metal.qlibrary.sample_shapes.n_gon import NGon
from qiskit_metal.qlibrary.qubits.star_qubit import StarQubit

In [ ]:
from qiskit_metal import MetalGUI

In [ ]:
design = designs.DesignPlanar()
gui = MetalGUI(design)
gui.rebuild() 

In [ ]:
StarQubit(design, 'Star')
gui.rebuild()